In [1]:
import keras
import pandas as pd
import numpy as np
import matchzoo as mz

Using TensorFlow backend.


## Read Data 

In [2]:
def read_data(path, include_label):
    def scan_file():
        with open(path) as in_file:
            next(in_file)  # skip header
            for l in in_file:
                yield l.strip().split('\t')
    if include_label:
        return [(qid, did, q, d, float(label)) for qid, q, _, _, did, d, label in scan_file()]
    else:
        return [(qid, did, q, d) for qid, q, _, _, did, d, _ in scan_file()]

train = read_data('data/WikiQA/WikiQACorpus/WikiQA-train.tsv', include_label=True)
predict  = read_data('data/WikiQA/WikiQACorpus/WikiQA-test.tsv', include_label=False)

## Preprocess Data

In [3]:
train_pack = mz.pack(train[:1000])
predict_pack = mz.pack(predict[:1000])

In [4]:
preprocessor = mz.preprocessors.DSSMPreprocessor()
preprocessor.fit(train_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 96/96 [00:00<00:00, 3247.86it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 1000/1000 [00:00<00:00, 3829.20it/s]
Fitting vocabulary unit.: 100%|██████████| 84935/84935 [00:00<00:00, 2845211.25it/s]


In [5]:
train_pack_processed = preprocessor.transform(train_pack)
predict_pack_processed = preprocessor.transform(predict_pack)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 96/96 [00:00<00:00, 5377.53it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 1000/1000 [00:00<00:00, 3206.69it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 108/108 [00:00<00:00, 5587.09it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit => WordHashingUnit: 100%|██████████| 992/992 [00:00<00:00, 3297.83it/s]


## Generate Data

In [6]:
train_generator = mz.DataGenerator(train_pack_processed, batch_size=32)

## Model Building

In [7]:
ranking_task = mz.tasks.Ranking()
ranking_task.metrics = [
    'mae', 'map', 'precision',
    mz.metrics.Precision(k=3),
    mz.metrics.DiscountedCumulativeGain(k=1),
    mz.metrics.DiscountedCumulativeGain(k=3),
    mz.metrics.DiscountedCumulativeGain(k=5),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=1),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=3),
    mz.metrics.NormalizedDiscountedCumulativeGain(k=5)
]
ranking_task.metrics

['mae',
 mean_average_precision(0),
 precision@1(0),
 precision@3(0),
 discounted_cumulative_gain@1(0),
 discounted_cumulative_gain@3(0),
 discounted_cumulative_gain@5(0),
 normalized_discounted_cumulative_gain@1(0),
 normalized_discounted_cumulative_gain@3(0),
 normalized_discounted_cumulative_gain@5(0)]

In [8]:
model = mz.models.DSSMModel()
model.params['task'] = ranking_task
model.params['input_shapes'] = preprocessor.context['input_shapes']
model.guess_and_fill_missing_params()
model.build()
model.compile()

Parameter "name" set to DSSMModel.


## Model Training

In [9]:
model.fit(*train_pack_processed.unpack(), epochs=2, batch_size=32)

Epoch 1/2
1000/1000 [==============================] - 1s 765us/step - loss: 0.0425 - mean_absolute_error: 0.0836
Epoch 2/2
1000/1000 [==============================] - 0s 436us/step - loss: 0.0317 - mean_absolute_error: 0.0758


In [10]:
model.fit_generator(train_generator, epochs=2, workers=4, use_multiprocessing=True)

Epoch 1/2
32/32 [==============================] - 1s 20ms/step - loss: 0.0255 - mean_absolute_error: 0.0613
Epoch 2/2
32/32 [==============================] - 1s 21ms/step - loss: 0.0222 - mean_absolute_error: 0.0531


### Training with Dynamic Data Generator

In [11]:
pp_no_hashing = mz.preprocessors.DSSMPreprocessor(with_word_hashing=False)
train_pack_processed_no_hashing = pp_no_hashing.fit_transform(train_pack)
word_hashing_unit = mz.processor_units.WordHashingUnit(pp_no_hashing.context['vocab_unit'].state['term_index'])
dynamic_data_generator = mz.data_generator.DynamicDataGenerator(word_hashing_unit.transform,
                                                                train_pack_processed_no_hashing)
model.fit_generator(dynamic_data_generator, epochs=2, workers=4, use_multiprocessing=True)

Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 96/96 [00:00<00:00, 4151.19it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 1000/1000 [00:00<00:00, 3526.08it/s]
Fitting vocabulary unit.: 100%|██████████| 84935/84935 [00:00<00:00, 2543558.77it/s]
Processing text_left with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 96/96 [00:00<00:00, 8111.47it/s]
Processing text_right with chain_transform of TokenizeUnit => LowercaseUnit => PuncRemovalUnit => StopRemovalUnit => NgramLetterUnit: 100%|██████████| 1000/1000 [00:00<00:00, 4009.97it/s]


Epoch 1/2
32/32 [==============================] - 1s 21ms/step - loss: 0.0192 - mean_absolute_error: 0.0476
Epoch 2/2
32/32 [==============================] - 1s 24ms/step - loss: 0.0168 - mean_absolute_error: 0.0440


## Model Predicting 

In [12]:
model.predict(predict_pack_processed[:10].unpack()[0])

array([[-0.12035733],
       [-0.01671004],
       [-0.09138029],
       [-0.10102976],
       [-0.03708177],
       [-0.09206843],
       [ 0.08697687],
       [ 0.08094203],
       [ 0.0192414 ],
       [-0.01353657]], dtype=float32)

## Model Evaluating

In [13]:
model.evaluate(*train_pack_processed.unpack())

1000/1000 [==============================] - 0s 137us/step


{'loss': 0.015919701620936394,
 'mean_absolute_error': 0.042832092851400376,
 'mean_average_precision(0)': 0.3333333333333333,
 'precision@1(0)': 0.3333333333333333,
 'precision@3(0)': 0.13194444444444445,
 'discounted_cumulative_gain@1(0)': 0.4808983469629878,
 'discounted_cumulative_gain@3(0)': 0.5338530547483661,
 'discounted_cumulative_gain@5(0)': 0.5338530547483661,
 'normalized_discounted_cumulative_gain@1(0)': 0.3333333333333333,
 'normalized_discounted_cumulative_gain@3(0)': 0.3333333333333333,
 'normalized_discounted_cumulative_gain@5(0)': 0.3333333333333333}

## Automation

In [14]:
train_pack.shuffle(inplace=True)

In [15]:
divider = int(len(train_pack) * 0.9)
train_pack_divided = train_pack[:divider]
test_pack_divided = train_pack[divider:]

In [16]:
director = mz.Director()
director.params['models'] = [
    mz.models.DenseBaselineModel(),
    mz.models.DSSMModel(),
    mz.models.ArcIModel(),
    mz.models.KNRMModel(),
    mz.models.ConvKNRMModel(),
]
director.params['evals_per_model'] = 5
director.params['train_pack'] = train_pack_divided
director.params['test_pack'] = test_pack_divided
director.params['task'] = ranking_task

In [17]:
trials = director.action(verbose=1)

Start tunning model #0 (total: 5).
Model class: <class 'matchzoo.models.dense_baseline_model.DenseBaselineModel'>
Finish tuning model #0 (total: 5)

Start tunning model #1 (total: 5).
Model class: <class 'matchzoo.models.dssm_model.DSSMModel'>
Finish tuning model #1 (total: 5)

Start tunning model #2 (total: 5).
Model class: <class 'matchzoo.models.arci_model.ArcIModel'>
Finish tuning model #2 (total: 5)

Start tunning model #3 (total: 5).
Model class: <class 'matchzoo.models.knrm_model.KNRMModel'>
Finish tuning model #3 (total: 5)

Start tunning model #4 (total: 5).
Model class: <class 'matchzoo.models.conv_knrm_model.ConvKNRMModel'>
Finish tuning model #4 (total: 5)



In [19]:
for trial in trials:
    print('Model Name:', trial['model_params']['name'])
    print("Hyper Parameter Sampled:" , trial['sampled_params'])
    print('Loss:', trial['loss'])
    print()

Model Name: DenseBaselineModel
Hyper Parameter Sampled: {'num_dense_layers': 1.0, 'num_dense_units': 281.0}
Loss: 315751.96875

Model Name: DenseBaselineModel
Hyper Parameter Sampled: {'num_dense_layers': 4.0, 'num_dense_units': 662.0}
Loss: 23790.8671875

Model Name: DenseBaselineModel
Hyper Parameter Sampled: {'num_dense_layers': 4.0, 'num_dense_units': 350.0}
Loss: 5561.4501953125

Model Name: DenseBaselineModel
Hyper Parameter Sampled: {'num_dense_layers': 1.0, 'num_dense_units': 879.0}
Loss: 29712.88671875

Model Name: DenseBaselineModel
Hyper Parameter Sampled: {'num_dense_layers': 4.0, 'num_dense_units': 389.0}
Loss: 11809.3515625

Model Name: DSSMModel
Hyper Parameter Sampled: {'dim_hidden': 165.0}
Loss: 0.06604287773370743

Model Name: DSSMModel
Hyper Parameter Sampled: {'dim_hidden': 172.0}
Loss: 0.0792979970574379

Model Name: DSSMModel
Hyper Parameter Sampled: {'dim_hidden': 207.0}
Loss: 0.05772292986512184

Model Name: DSSMModel
Hyper Parameter Sampled: {'dim_hidden': 433.

## Misc

### Load Embedding Matrix

In [20]:
model = mz.models.ArcIModel()
model.params['task'] = ranking_task
term_index = preprocessor.context['vocab_unit'].state['term_index']
model.params['embedding_input_dim'] = len(term_index) + 1
model.guess_and_fill_missing_params()
model.build()
model.compile()

Parameter "name" set to ArcIModel.
Parameter "input_shapes" set to [(30,), (30,)].
Parameter "embedding_trainable" set to True.


In [21]:
vocab = preprocessor.context['vocab_unit'].state['term_index']
embedding_matrix = np.random.uniform(
    -0.2, 0.2,
    (model.params['embedding_input_dim'], model.params['embedding_output_dim'])
)
model.load_embedding_matrix(embedding_matrix)